In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [24]:
# Load mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./bert_finetuned")
tokenizer = AutoTokenizer.from_pretrained("./bert_finetuned")

In [25]:
# Load MultiLabelBinarizer để giải mã nhãn
mlb = MultiLabelBinarizer()
mlb.classes_ = np.array(['fiction', 'romance', 'young adult', 'fantasy', 'contemporary'])

In [26]:
# Hàm inference
def predict_tags(descriptions, threshold=0.5):
    """
    Dự đoán nhãn cho một hoặc nhiều đoạn mô tả.
    
    Args:
        descriptions (Union[str, List[str]]): Một đoạn mô tả hoặc danh sách các mô tả.
        threshold (float): Ngưỡng dự đoán cho các nhãn.
        
    Returns:
        List[List[str]]: Danh sách các nhãn dự đoán cho mỗi mô tả.
    """
    if isinstance(descriptions, str):
        descriptions = [descriptions]
    
    # Tokenize mô tả
    inputs = tokenizer(descriptions, truncation=True, padding=True, max_length=128, return_tensors="pt")
    
    # Dự đoán logits
    model.eval()
    with torch.no_grad():
        logits = model(**inputs).logits
    
    # Áp dụng sigmoid để tính xác suất
    probs = torch.sigmoid(logits).numpy()
    
    # Lấy nhãn dựa trên ngưỡng
    predicted_labels = [
        [mlb.classes_[i] for i, prob in enumerate(prob_vector) if prob > threshold]
        for prob_vector in probs
    ]
    
    return predicted_labels

In [27]:
# Danh sách các mô tả
descriptions = [
    "I can see ghosts. I can talk to ghosts. And, if necessary, I can kick some serious ghost butt.Susannah Simon has an unearthly gift: she's a Mediator. She must help trapped and tortured souls move on to the next world. But not all spooks want her guidance or want to go quietly - that's when Suze has to get tough. Then she meets Jesse, a drop-dead-gorgeous ghost who just happens to haunt her bedroom. It's hard to boot a guy into the afterlife when he makes your heart beat faster - even though he's been dead for 150 years...",
    "Conspiracy thrillers have tackled Da Vinci, Atlantis and the Pyramids - but never Stonehenge. Until now. With a concept to rival the very biggest books in the genre, this novel will dazzle anyone interested in one of the world's most fascinating mysteries.Eight days before the summer solstice, a man is butchered in a blood-freezing sacrifice on the ancient site of Stonehenge before a congregation of robed worhsippers. Within hours, one of the world's foremost treasure hunters has shot himself in his country mansion. And to his estranged son, young archaeologist Gideon Chase, he leaves a cryptic letter . . .Teaming up with an intrepid Wiltshire policewoman, Gideon soon exposes a secret society - an ancient international legion devoted for thousands of years to Stonehenge. With a charismatic and ruthless new leader at the helm, the cult is now performing ritual human sacrifices in a terrifying bid to unlock the secret of the stones.Packed with codes, symbology, relentless suspense, and fascinating detail about the history of one of the world's most mysterious places, The Stonehenge Legacy is a blockbuster thriller to rival the very best of Dan Brown.",
    "From the author of the international bestseller Unraveling Oliver comes a truly outstanding (Crime by the Book), brilliantly plotted, utterly immersive novel lauded by A.J. Finn#1 New York Times bestselling author of The Woman in the Windowas extraordinarycrackles and snaps like a bonfire on a winters night.My husband did not mean to kill Annie Doyle, but the lying tramp deserved it.  On the surface, Lydia Fitzsimons's life seems idyllic. Her husband, Andrew, is a prosperous, respected judge; they live in the spacious, comfortable, well-appointed house where she was raised. And she is utterly, obsessively devoted to her son, Laurenceher adored only child, her pride and joy.  But Andrew's murder of Annie Doyle, accidental or not, sets into motion a dark downward spiral. It doesnt take long for Laurence to suspect that something is very, very wrong in the Fitzsimons householdand he is determined to discover the truth.  For fans of Ruth Ware and Gillian Flynn, Lying in Wait is a devastating psychological thriller...an exquisitely uncomfortable, utterly captivating reading experience (Publishers Weekly, starred review).",
    "When I met Kiersten, time stood still. It was as if every cell in my body screamed HER. I couldn't look away, I couldn't walk away, I couldn't even move. My other half, my soul mate, was staring right at me. I had to have her. I had to make her mine. But I felt selfish doing that--knowing that I may not have endless amounts of time to give her...and even now that every things' over with and I've been given that time back. Nothing is ever certain in this life. Which is why I'm about to do something crazy, impulsive, daring. After all, what type of person would I be if I didn't take my own advice?I'm jumping off the cliff, I'm living fearlessly.This is the continuation of Wes and Kiersten's story.",
    "Cynthia Eden invites you into the world of the Others - supernatural creatures who are sexy, powerful, and more than human in every way...No one that deadly should look that good. Hybrid demon Zane Wynter specializes in tracking the most lethal criminals, whether human or Other. But Jana Carter is nothing like his previous targets. She's an Ignitor, able to conjure and control fire. She's also sexy beyond belief - and used to playing dirty.Jana knows she's being set up. She even knows who's responsible: Project Perseus, a secretive group that plans to rid the world of paranormals by any means necessary. From Louisiana's sultry swamps to its desolate cemeteries, Jana and Zane must race to destroy the mastermind behind Perseus before their world goes up in flames - and with it, a desire that burns too hot to resist."
]

# Dự đoán nhãn cho các mô tả
predictions = predict_tags(descriptions)

# Hiển thị kết quả
for i, (description, tags) in enumerate(zip(descriptions, predictions)):
    print(f"Description {i + 1}: {description}")
    print(f"Predicted Tags: {tags}")
    print("-" * 50)

Description 1: I can see ghosts. I can talk to ghosts. And, if necessary, I can kick some serious ghost butt.Susannah Simon has an unearthly gift: she's a Mediator. She must help trapped and tortured souls move on to the next world. But not all spooks want her guidance or want to go quietly - that's when Suze has to get tough. Then she meets Jesse, a drop-dead-gorgeous ghost who just happens to haunt her bedroom. It's hard to boot a guy into the afterlife when he makes your heart beat faster - even though he's been dead for 150 years...
Predicted Tags: ['romance', 'young adult', 'fantasy', 'contemporary']
--------------------------------------------------
Description 2: Conspiracy thrillers have tackled Da Vinci, Atlantis and the Pyramids - but never Stonehenge. Until now. With a concept to rival the very biggest books in the genre, this novel will dazzle anyone interested in one of the world's most fascinating mysteries.Eight days before the summer solstice, a man is butchered in a bl